In [6]:
import requests
from lxml import etree
from multiprocessing.dummy import Pool #线程池. selenium
import time

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
}
def get_request(url):
    response = requests.get(url,headers=headers)
    response.encoding = 'utf-8'
    xpath_now = etree.HTML(response.text)
    xpath_now2 = xpath_now.xpath('//div[@class="content"]/span[1]')
    xpath_out = []
    for i in xpath_now2:
        text1 = i.xpath("string(.)").strip()
        xpath_out.append(text1)
    return xpath_out

In [13]:
#使用普通循环爬取方式对比
url = 'https://www.qiushibaike.com/text/page/%d/'
urls = []
num = int(input('请输入爬取页数：'))
for i in range(1,num+1):
    urls.append(format(url%i))
#步骤：1.实例化线程池
time_1 = time.time()
response_text_list = list(map(get_request,urls)) #普通处理
"""
for i in range(num):
    for j in range(len(response_text_list[i])):
        print('  第{0}页-第{1}个趣事：{2}'.format(i+1,j+1,response_text_list[i][j]))
"""
time_2 = time.time()
print("异步获取所耗时间戳：{}".format(time_2-time_1))

请输入爬取页数：13
异步获取所耗时间戳：2.7076704502105713


In [15]:
#基于线程池的异步爬取方法
url = 'https://www.qiushibaike.com/text/page/%d/'
urls = []
num = int(input('请输入爬取页数：'))
for i in range(1,num+1):
    urls.append(format(url%i))
    
#实例化线程池
pool = Pool(13)
time_1 = time.time()
response_text_list = pool.map(get_request,urls) #参数1是函数，参数2是可迭代对象. 异步处理
"""
for i in range(num):
    for j in range(len(response_text_list[i])):
        print('  第{0}页-第{1}个趣事：{2}'.format(i+1,j+1,response_text_list[i][j]))
"""
time_2 = time.time()
print("异步获取所耗时间戳：{}".format(time_2-time_1))

请输入爬取页数：13
异步获取所耗时间戳：0.37599945068359375


In [10]:
#使用协程解决（单线程+多任务异步协程）
#协程： 可以把协程当作一个特别的函数，如果一个函数的定义被async所修饰,就不会立即执行，而是返回协程对象
from time import sleep
import nest_asyncio  #针对jupyternotebook错误情况~
import asyncio
import time
nest_asyncio.apply()

urls = [
    'http://localhost:5000/bobo',
    'http://localhost:5000/jay',
    'http://localhost:5000/tom'
]

#回调函数
def callback1(task):
    print('callback —— now')
    print(task.result())

#待执行函数中不可以使用不支持异步模块的代码
async def get_requests1(url):
    print('正在请求：',url)
    await asyncio.sleep(2) #必须要用await 执行阻塞操作。
    print('请求结束：',url)
    return ('hello ~ ' + url)

tasks = [] #用于放封装了的任务对象

for i in urls:
    c = get_requests1(i) #接收协程对象

    #任务对象(task)： 对协程对象作进一步的封装！ 比如显示协程运行情况
    task = asyncio.ensure_future(c)
    task.add_done_callback(callback1) #绑定回调函数
    tasks.append(task)

#注册到事件循环对象中,事件循环对象中可以放很多任务对象去执行。 异步体现在开始时是顺序执行任务对象，而一个任务事件发生阻塞不会等待，而是继续执行
#下一个对象
time1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks)) #每一个任务对象，进行挂起操作。在阻塞时就自动挂起
time2 = time.time()
print(time2-time1)




正在请求： http://localhost:5000/bobo
正在请求： http://localhost:5000/jay
正在请求： http://localhost:5000/tom
请求结束： http://localhost:5000/bobo
请求结束： http://localhost:5000/jay
请求结束： http://localhost:5000/tom
callback —— now
hello ~ http://localhost:5000/bobo
callback —— now
hello ~ http://localhost:5000/jay
callback —— now
hello ~ http://localhost:5000/tom
2.0040602684020996
